In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
NN = tf.keras.models.load_model("./model/NN")
CNN = tf.keras.models.load_model("./model/CNN")

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
data = pd.read_csv('factor_pass9.csv', index_col=0)
data['continuous_count'] = data.groupby((data['evebit'] != data['evebit'].shift(1)).cumsum()).cumcount() + 1
data = data.reindex(columns=data.columns.tolist()[:-2] + ['continuous_count', 'return'])
data.dropna(inplace=True)

insample_data = data.loc[data.loc[:, 'date'] < "2022-01-01", :]
other_info_insample = insample_data.iloc[:, :3]
insample_data_sorted = insample_data.sort_values('date')
insample_data_sorted['rank'] = insample_data_sorted['date'].rank()
sum_of_weight = (insample_data_sorted['rank']).sum()
insample_data_sorted['weight'] = insample_data_sorted['rank'] / sum_of_weight
outdsample_data = data.loc[data.loc[:, 'date'] >= "2022-01-01", :]

X = insample_data_sorted.iloc[:, 3:-3]
y = insample_data_sorted.iloc[:, -3]
other_info_outsample_test = outdsample_data.iloc[:, :3]
X_outsample_test = outdsample_data.iloc[:, 3:-1]
y_outsample_test = outdsample_data.iloc[:, -1]

In [5]:
from sklearn.preprocessing import StandardScaler

scaler_X = StandardScaler()

X_scaled = scaler_X.fit_transform(X)

X_scaled = pd.DataFrame(X_scaled, index=X.index, columns=X.columns)

X_outsample_test_scaled = scaler_X.transform(X_outsample_test)

In [6]:
y_info = pd.concat([other_info_insample, y], axis=1)
y_info

,id,name,date,return
20,338,潍柴动力,2016-02-01,0.016925
21,338,潍柴动力,2016-02-02,-0.008322
22,338,潍柴动力,2016-02-03,0.012587
23,338,潍柴动力,2016-02-04,-0.008287
24,338,潍柴动力,2016-02-05,-0.012535
...,...,...,...,...
16732,601689,拓普集团,2021-12-27,-0.006483
16733,601689,拓普集团,2021-12-28,-0.013609
16734,601689,拓普集团,2021-12-29,0.020601
16735,601689,拓普集团,2021-12-30,-0.018519


In [7]:
def cal_weight(df):
    df.loc[:, "return"] = df.loc[:, "return"] - df.loc[:, "return"].min()
    df.loc[:, "weight"] = df.loc[:, "return"]/df.loc[:, "return"].sum() if df.loc[:, "return"].sum() != 0 else 1/8
    return df

y = y_info.groupby('date').apply(cal_weight)["weight"]

In [8]:
y_info_out = pd.concat([other_info_outsample_test, y_outsample_test], axis=1)
y_outsample_test_weight = y_info_out.groupby('date').apply(cal_weight)["weight"]

In [12]:
from sklearn.metrics import mean_squared_error

In [17]:
NN.predict(X_outsample_test_scaled)

68/68 [==============================] - 0s 698us/step


array([[0.11796592],
       [0.11779053],
       [0.11779595],
       ...,
       [0.11781059],
       [0.11821248],
       [0.11776026]], dtype=float32)

In [19]:
NN.predict(X_scaled).T, y

378/378 [==============================] - 0s 641us/step


0.00010720067050535757